In [65]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

loader = PyPDFLoader("Zain Ali Shah.pdf")
data = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = splitter.split_documents(data)

# Embeding
db = Chroma.from_documents(documents, OpenAIEmbeddings())
# export db as retriver
retriever = db.as_retriever()


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F6BD4B4620>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F6BC1C1A60>, root_client=<openai.OpenAI object at 0x000001F6AA8044D0>, root_async_client=<openai.AsyncOpenAI object at 0x000001F6BCA01BE0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [ ]:
# Design Prompt Template
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. Think step by step before providing a detailed answer.
I will tip you $1000 if the user is satisfied with your answer.

Context:
{context}

Question: {input}
""")

prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only provided context. Think step by step before providing a detailed answer.\nI will tip you $1000 if the user satisfy with your answer.\n\n<context>\n{context}\n<context/>\n\nQuestion: {input}\n\n'), additional_kwargs={})])

In [ ]:
# Chains
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only provided context. Think step by step before providing a detailed answer.\nI will tip you $1000 if the user satisfy with your answer.\n\n<context>\n{context}\n<context/>\n\nQuestion: {input}\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F6BD4B4620>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F6BC1C1A60>, root_client=<openai.OpenAI object at 0x000001F6AA8044D0>, root_async_client=<openai.AsyncOpenAI

In [69]:
# Retriver chain
from langchain.chains.retrieval import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever=retriever,combine_docs_chain=document_chain)


response = retriever_chain.invoke({"input":"In which company is zain currently working"})
response["answer"]

'To determine the company Syed Zain Ali Shah is currently working for, we need to identify the most recent position he holds. \n\n1. According to the context, Zain is currently working as a Frontend Engineer at Kodexo Labs.\n2. His tenure at Kodexo Labs started in January 2025 and is listed as "Present," indicating he is currently employed there.\n3. His previous role was at Techsol LLC as a Full Stack Developer, which ended in December 2024.\n\nTherefore, based on the latest information provided, Syed Zain Ali Shah is currently working at Kodexo Labs as a Frontend Engineer.'